# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")
import pymongo 
import pandas as pd
import numpy as np
datos = pymongo.MongoClient()

In [2]:
pd.options.display.max_columns = None

In [3]:
datos.list_database_names()

['Powerlifting', 'admin', 'alojamientos', 'companies', 'config', 'local']

In [4]:
mydb = datos["companies"]
datos = mydb.datos

In [5]:
df = pd.read_json("../companies.json", lines=True)

In [6]:
df.head(1)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,{'$oid': '52cdef7c4bab8bd675297d8a'},Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,10.0,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,{'$date': 1180075887000},2013-12-08 07:15:44+00:00,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN


In [7]:
df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots',
       'external_links', 'partners', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'ipo'],
      dtype='object')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [8]:
# Your Code
ej_1 = pd.DataFrame(list(datos.find({"name":"Babelgum"})))
ej_1

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,deadpooled_month,deadpooled_day,deadpooled_url,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7c4bab8bd675297da0,Babelgum,babelgum,http://www.crunchbase.com/company/babelgum,http://babelgum.com,http://babelgum.com/blog,http://feeds.feedburner.com/Babelgum,Babelgum,games_video,None,2007,3,1,2013,1,9,,"iptv, web2ireland",,info@babelgum.com,,,Sat Jun 09 08:15:21 UTC 2007,Wed Oct 16 06:30:25 UTC 2013,<p>Babelgum is an integrated web and mobile vi...,"{'available_sizes': [[[135, 102], 'assets/imag...","[{'name': 'Babelgum', 'permalink': 'babelgum'}]","[{'is_past': False, 'title': 'CEO', 'person': ...","[{'competitor': {'name': 'Joost', 'permalink':...",[],$13.2M,"[{'id': 17, 'round_code': 'a', 'source_url': '...",[],None,[],"[{'description': '', 'address1': '', 'address2...",[],None,[],[],[],[]


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [9]:
# Your Code
ej_2 = pd.DataFrame(list(datos.find({"number_of_employees":{"$gt": 5000}}).sort("number_of_employees", -1)[:20]))
ej_2.name.unique()

array(['Siemens', 'IBM', 'Toyota', 'PayPal',
       'Nippon Telegraph and Telephone Corporation',
       'Samsung Electronics', 'Accenture', 'Tata Consultancy Services',
       'Flextronics International', 'Safeway', 'Sony', 'LG', 'Ford',
       'Boeing', 'Digital Equipment Corporation', 'Nokia',
       'MItsubishi Electric', 'Comcast', 'Bertelsmann'], dtype=object)

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [10]:
# Your Code
cond1 = {"founded_year":{"$gte": 2000}}
cond2 = {"founded_year":{"$lte": 2005}}
ej_3 = pd.DataFrame(list(datos.find({"$and":[cond1, cond2]}, {"name":1,"founded_year":1,"_id":0})))
ej_3.head()

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [11]:
# Your Code
cond1 = {"founded_year":{"$lt": 2010}}
cond2 = {"ipo.valuation_amount":{"$gt": 100000000}}
ej_4 = pd.DataFrame(list(datos.find({"$and":[cond1, cond2]}, {"name":1,"ipo":1,"_id":0})))
ej_4.head()

,name,ipo
0,Facebook,"{'valuation_amount': 104000000000, 'valuation_..."
1,Twitter,"{'valuation_amount': 18100000000, 'valuation_c..."
2,Yelp,"{'valuation_amount': 1300000000, 'valuation_cu..."
3,LinkedIn,"{'valuation_amount': 9310000000, 'valuation_cu..."
4,Amazon,"{'valuation_amount': 100000000000, 'valuation_..."


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [12]:
# Your Code
cond3 = {"number_of_employees":{"$lt": 1000}}
cond4 = {"founded_year":{"$lt": 2004}}
ej_5 = pd.DataFrame(list(datos.find({"$and":[cond3, cond4]}, {"name":1,"_id":0}).sort("number_of_employees", -1)[:10]))
ej_5

,name
0,Infinera Corporation
1,NorthPoint Communications Group
2,888 Holdings
3,Forrester Research
4,SonicWALL
5,Webmetrics
6,Cornerstone OnDemand
7,MySpace
8,Mozilla
9,ZoomInfo


### 6. All the companies that don't include the `partners` field.

In [13]:
# Your Code
ej_6 = pd.DataFrame(list(datos.find({"partners": {'$exists': False}}, {"name":1,"_id":0})))
ej_6.head()

""


### 7. All the companies that have a null type of value on the `category_code` field.

In [14]:
# Your Code
ej_7 = pd.DataFrame(list(datos.find({"category_code":{"$type":"null"}}, {"name":1,"_id":0})))
ej_7.head()

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [15]:
# Your Code
cond5 = {"number_of_employees":{"$gte": 100}}
cond6 = {"number_of_employees":{"$lt": 1000}}
ej_8 = pd.DataFrame(list(datos.find({"$and":[cond5, cond6]}, {"name":1,"number_of_employees":1 ,"_id":0})))
ej_8.head()

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [16]:
# Your Code
ej_9 = pd.DataFrame(list(datos.find((),{"name":1,"_id":0}).sort("ipo", -1)))
ej_9.head()

,name
0,GREE
1,Facebook
2,Amazon
3,Twitter
4,Groupon


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [17]:
# Your Code
ej_10 = pd.DataFrame(list(datos.find((),{"name":1,"_id":0}).sort("number_of_employees", -1)[:10]))
ej_10

,name
0,Siemens
1,IBM
2,Toyota
3,PayPal
4,Nippon Telegraph and Telephone Corporation
5,Samsung Electronics
6,Accenture
7,Tata Consultancy Services
8,Flextronics International
9,Safeway


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [19]:
# Your Code
ej_11 = pd.DataFrame(list(datos.find(({"founded_month":{"$gt":6}}),{"name":1,"_id":0}))[:1000])
ej_11.head()

,name
0,Wetpaint
1,Zoho
2,Digg
3,Omnidrive
4,eBay


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [21]:
# Your Code
cond7 = {"founded_year":{"$lt": 2000}}
cond8 = {"acquisition.price_amount":{"$gt": 10000000}}
ej_12 = pd.DataFrame(list(datos.find({"$and":[cond7, cond8]}, {"name":1,"_id":0})))
ej_12.head()

,name
0,Postini
1,SideStep
2,Recipezaar
3,PayPal
4,Snapfish


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [22]:
# Your Code
ej_13 = pd.DataFrame(list(datos.find({"acquisition.acquired_year":{"$gt":2010}},
               {"name":1,"acquisition.price_amount":1,"_id":0}).sort("acquisition.price_amount",-1)))
ej_13.head()

,name,acquisition
0,T-Mobile,{'price_amount': 39000000000}
1,Goodrich Corporation,{'price_amount': 18400000000}
2,LSI,{'price_amount': 6600000000}
3,National Semiconductor,{'price_amount': 6500000000}
4,Ariba,{'price_amount': 4300000000}


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [23]:
# Your Code
ej_14 = pd.DataFrame(list(datos.find((),{"name":1,"founded_year":1,"_id":0}).sort("founded_year", -1)))
ej_14.head()

,name,founded_year
0,Fixya,2013.0
1,Wamba,2013.0
2,Advaliant,2013.0
3,Fluc,2013.0
4,iBazar,2013.0


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [24]:
# Your Code
ej_15 = pd.DataFrame(list(datos.find({"founded_day":{"$lte":7}},
                        {"name":1,"acquisition.price_amount":1,"_id":0})
                          .sort("acquisition.price_amount",-1))[:10])
ej_15

,name,acquisition
0,Netscape,{'price_amount': 4200000000}
1,PayPal,{'price_amount': 1500000000}
2,Zappos,{'price_amount': 1200000000}
3,Alibaba,{'price_amount': 1000000000}
4,Postini,{'price_amount': 625000000}
5,Danger,{'price_amount': 500000000}
6,Clearwell Systems,{'price_amount': 410000000}
7,PrimeSense,{'price_amount': 345000000}
8,Amobee,{'price_amount': 321000000}
9,BlueLithium,{'price_amount': 300000000}


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [25]:
# Your Code
cond9 = {"number_of_employees":{"$gt":4000}}
cond10 = {"category_code":"web"}
ej_16 = pd.DataFrame(list(datos.find({"$and":[cond9,cond10]},{"name":1,"_id":0}).sort("number_of_employees",-1)))
ej_16.head()

,name
0,Experian
1,eBay
2,Yahoo!
3,Rakuten
4,Los Angeles Times Media Group


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [26]:
# Your Code
cond11 = {"acquisition.price_amount":{"$gt":10000000}}
cond12 = {"acquisition.price_currency_code":"EUR"}
ej_17 = pd.DataFrame(list(datos.find({"$and":[cond11,cond12]},{"name":1,"_id":0})))
ej_17

,name
0,ZYB
1,Apertio
2,Greenfield Online
3,Webedia
4,Wayfinder
5,Tuenti Technologies
6,BioMed Central


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [27]:
# Your Code
ej_18 = pd.DataFrame(list(datos.find(({"acquisition.acquired_month":{"$lte":3}}),{"name":1,"acquisition":1,"_id":0}))[:10])
ej_18

,name,acquisition
0,Kyte,"{'price_amount': None, 'price_currency_code': ..."
1,NetRatings,"{'price_amount': 327000000, 'price_currency_co..."
2,blogTV,"{'price_amount': None, 'price_currency_code': ..."
3,Livestream,"{'price_amount': None, 'price_currency_code': ..."
4,iContact,"{'price_amount': 169000000, 'price_currency_co..."
5,Coghead,"{'price_amount': None, 'price_currency_code': ..."
6,Dailymotion,"{'price_amount': 168000000, 'price_currency_co..."
7,Netvibes,"{'price_amount': None, 'price_currency_code': ..."
8,Flickr,"{'price_amount': None, 'price_currency_code': ..."
9,BabyCenter,"{'price_amount': None, 'price_currency_code': ..."


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code